In [ ]:
def quadrilateral(n:int, m:int, P1:list, P2:list, P3:list, P4:list, fill=0):
    """
    Funkcja rysujaca czworokat na polu o podanych wymiarach [n x m].
    O punktach kolejnk: P1, P2, P3 i P4, zawierajace wspolrzedne tych punktow
    Z mozliwoscia rysowania czworokatu z oraz bez wypelnienia.
    Args:
        n (int): wymiar n zwracanej tablicy rgb
        m (int): wymiar m zwracanej tablicy rgb
        P1 (list): wspolrzedne punktu P1
        P2 (list): wspolrzedne punktu P2
        P3 (list): wspolrzedne punktu P3
        P4 (list): wspolrzedne punktu P4
        fill (int, optional): Wartosc oznaczajaca czy rysowany czworokat ma byc wypelniony czy ma byc rysowany tylko obwod.
        Dla wartosci 1 rysowana jest wypelniona elipsa.
        Dla wartosci 0 rysowany jest obwod elipsy i jest to wartosc domyslna.
    """
    
    